In [19]:
from komoran3py import KomoranPy
import nltk

In [20]:
ko = KomoranPy()
ko.set_user_dictionary('./user_dictionary.txt')

In [21]:
# 형태소 분석한 파일이랑 원본 파일 불러오기
# 서로 비교하여 잡히지 않은 단어 추출 하여 리스트에 튜플로 저장
# 저장된 리스트 피클로 저장 

In [22]:
def call_file_list(path) : 
    """
        path : 파일 경로
    """
    from os import listdir
    path_dir = path
    file_list = listdir(path_dir)
    return file_list

In [23]:
def openlog( bj_code, filename, platform = 'twitch_live_chat') : 
    """
    '../../data/{0}/{1}/{2}'.format(platform, bj_code, filename)
    platform : 플랫폼 폴더명 (기본 : twitch_live_chat)
    bj_code : bj 폴더명
    filename : 로그 파일이름
    """
    chatlog = '../../data/{0}/{1}/{2}'.format(platform, bj_code, filename)
    with open(chatlog) as f : 
        chat = f.readlines()
    return chat

In [24]:
# 전처리 후 문장들을 2중 리스트로 반환
def preprocessing_chat_tolist(chat, num = 3) : 
    """ 
        chat : chat에는 채팅로그 넣기
        num : 그룹 선택 / 안주면 기본값 3 (0 = 전체, 1 = 시간(방송기준) 2 = ID 3 = 채팅)
    """
    import re
    text = chat
    my = re.compile('\[([0-9:]*)\] <(\S*[ ]*\S*)> (\w.*)')
    word_tolist = []
    for line in text:
        mytext = my.search(line)
        if mytext :
            word_tolist.append([mytext.group(num)])
    return word_tolist

In [29]:
# 형태소 분석된 거를 원래 문장단위로 리스트로 묶어서 리턴 / 이중 리스트
def tokenize_tolist(word_list_tolist) : 
    """
        word_list : 전처리, 리스트화 된 채팅 로그 (preprocessing_tolist 리턴값)
    """
    tokens_ko_tolist = []
    line_list = []
    for word_list in word_tolist : 
        for word in word_list : 
            for i in range(len(ko.pos(word))) : 
                line_list.append(ko.pos(word)[i][0])
                if i+1 == len(ko.pos(word)) : 
                    tokens_ko_tolist.append(line_list)
                    line_list = []
    return tokens_ko_tolist

In [26]:
# preprocessing_tolist 리턴값과 tokenize_tolist 리턴값 넣으면, 추가되거나 너무 작게
# 쪼개진 형태소와 문장이 튜플로 묶인후, 그 튜플을 모은 리스트가 반환
def matching(word_tolist, tokens_ko_tolist) : 
    """
        word_list : 전처리, 리스트화 된 채팅 로그 (preprocessing_tolist 리턴값)
    """
    matching_list = []
    for i , value in enumerate(word_list):
        try : 
            for target in tokens_ko[i]:
                if target not in value[0]:
                    matching_list.append(target, value)
        except Exception as e : 
            print(e, target, value)
    return matching_list

In [27]:
# 디렉토리 체크해서 디렉토리 없으면 디렉토리 만들어줌
def mk_dir(dirpath) : 
    """
        dirpath : 체크할 폴더의 상대경로
    """
    import os
    dirname = dirpath
    if not os.path.isdir(dirname):
        os.mkdir(dirname)


In [28]:
def pickle_data2(name, bj_code, data) : 
    """
        name : 파일이름 지정
        data : data는 저장할 data 
    """
    import pickle
    with open('./matching_words/{0}/{1}.pickle'.format(bj_code, name), 'wb') as f : 
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [8]:
folder_list = call_file_list('../../../data/twitch_live_chat')

In [11]:
for folder in folder_list : 
    file_list = call_file_list('../../../data/twitch_live_chat/%s'%folder) # 각 bj폴더

    for file in file_list : 
        log = openlog('../../../data/twitch_live_chat/%s/%s'%(folder,file)) # bj폴더 내에서 로그 읽음
        word_tolist = preprocessing_chat_tolist(log) # word_tolist 반환
        tokens_ko_tolist = tokenize_tolist(word_tolist) # token_ko_tolist 반환
        matching_list = matching(word_tolist, tokens_ko_tolist)
        mk_dir('./matching_words/%s'%folder) # matching폴더 없으면 생성해줌
        pickle_data('matching_{}'.format(file[:-4]), folder, matching_list  )

../../../data/twitch_live_chat/#beyou0728/2018-12-04_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-05_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-06_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-07_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-08_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-09_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-10_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-11_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-12_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-14_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-15_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-16_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-17_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-12-18_#beyou0728.log
../../../data/twitch_live_chat/#beyou0728/2018-1